<a href="https://colab.research.google.com/github/BrandonSin/CPSC501Assignment4/blob/master/Heart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
 try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
import functools
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow_core.python.keras import regularizers
from tensorflow.keras import layers
import pandas as pd

In [58]:
from google.colab import files
uploaded = files.upload()


Saving heart_test.csv to heart_test.csv
Saving heart_train.csv to heart_train.csv


In [0]:
train_file = "heart_train.csv"
test_file = "heart_test.csv"

In [60]:
!head {train_file}

sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
160,12,5.73,23.11,Present,49,25.3,97.2,52,1
144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
170,7.5,6.41,38.03,Present,51,31.99,24.26,58,1
134,13.6,3.5,27.78,Present,60,25.99,57.34,49,1
132,6.2,6.47,36.21,Present,62,30.77,14.14,45,0
142,4.05,3.38,16.2,Absent,59,20.81,2.62,38,0
114,4.08,4.59,14.6,Present,62,23.11,6.72,58,1
114,0,3.83,19.4,Present,49,24.86,2.49,29,0


In [61]:
!head {test_file}

sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
160,12,5.73,23.11,Present,49,25.3,97.2,52,1
144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
170,7.5,6.41,38.03,Present,51,31.99,24.26,58,1
134,13.6,3.5,27.78,Present,60,25.99,57.34,49,1
132,6.2,6.47,36.21,Present,62,30.77,14.14,45,0
142,4.05,3.38,16.2,Absent,59,20.81,2.62,38,0
114,4.08,4.59,14.6,Present,62,23.11,6.72,58,1
114,0,3.83,19.4,Present,49,24.86,2.49,29,0


In [0]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [0]:
class PackNumericFeatures(object):
  def __init__(self,names):
    self.names = names
  
  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features


    return features, labels

In [0]:
def get_dataset(file, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file,
      batch_size = 50,
      label_name=LABEL_COLUMN,
      na_value ="?",
      num_epochs=1,
      ignore_errors=True,
      **kwargs)
  return dataset

In [0]:
LABEL_COLUMN = 'chd'
LABELS = [0, 1]

CATEGORIES = {'famhist': ['Present', 'Absent']}

In [0]:
train_data = "heart_train.csv"
test_data = "heart_test.csv"

SELECT_COLUMNS = ['sbp','tobacco','ldl','adiposity','famhist', 'typea','obesity','alcohol','age','chd']
raw_train_data = get_dataset(train_data, select_columns=SELECT_COLUMNS)
raw_test_data = get_dataset(test_data, select_columns=SELECT_COLUMNS)

In [0]:
train_batch, label_batch = next(iter(raw_train_data))
test_batch, label_batch = next(iter(raw_test_data))

In [0]:
NUMERIC_FEATURES = ['sbp','tobacco','ldl','adiposity', 'typea','obesity','alcohol','age']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [0]:
train_batch, label_batch = next(iter(packed_train_data))
test_batch, label_batch = next(iter(packed_test_data))

In [93]:
import pandas as pd
desc = pd.read_csv(train_data)[NUMERIC_FEATURES].describe()
desc

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age
count,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000
mean,138.326840,3.635649,4.740325,25.406732,53.103896,26.044113,17.044394,42.816017
std,20.496317,4.593024,2.070909,7.780699,9.817534,4.213680,24.481059,14.608956
min,101.000000,0.000000,0.980000,6.740000,13.000000,14.700000,0.000000,15.000000
25%,124.000000,0.052500,3.282500,19.775000,47.000000,22.985000,0.510000,31.000000
50%,134.000000,2.000000,4.340000,26.115000,53.000000,25.805000,7.510000,45.000000
75%,148.000000,5.500000,5.790000,31.227500,60.000000,28.497500,23.892500,55.000000
max,218.000000,31.200000,15.330000,42.490000,78.000000,46.580000,147.190000,64.000000


In [0]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])
def normalize_numeric_data(data, mean, std):
  return (data-mean)/std

In [0]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]

In [0]:
train_batch['numeric']

In [0]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(train_batch).numpy()

In [0]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [0]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [0]:
model = tf.keras.Sequential([
  preprocessing_layer,
  layers.Dense(256, activation='relu'),
  layers.Dropout(0.7),
   layers.Dense(128, activation='relu'),
  layers.Dropout(0.5),
   layers.Dense(128, activation='selu'),
  layers.Dropout(0.3),
  layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [109]:

print("--Train--")
model.fit(train_data, epochs=300)

test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy * 100))

--Train--
Epoch 1/300
10/10 [==============================] - 0s 16ms/step - loss: 0.4020 - accuracy: 0.7965
Epoch 2/300
10/10 [==============================] - 0s 11ms/step - loss: 0.3902 - accuracy: 0.7944
Epoch 3/300
10/10 [==============================] - 0s 9ms/step - loss: 0.3851 - accuracy: 0.7922
Epoch 4/300
10/10 [==============================] - 0s 10ms/step - loss: 0.4085 - accuracy: 0.8095
Epoch 5/300
10/10 [==============================] - 0s 10ms/step - loss: 0.4015 - accuracy: 0.8247
Epoch 6/300
10/10 [==============================] - 0s 10ms/step - loss: 0.3999 - accuracy: 0.8030
Epoch 7/300
10/10 [==============================] - 0s 9ms/step - loss: 0.3665 - accuracy: 0.8117
Epoch 8/300
10/10 [==============================] - 0s 9ms/step - loss: 0.4119 - accuracy: 0.8052
Epoch 9/300
10/10 [==============================] - 0s 8ms/step - loss: 0.3872 - accuracy: 0.8160
Epoch 10/300
10/10 [==============================] - 0s 9ms/step - loss: 0.4199 - accuracy: 0

In [108]:
# Show some results
predictions = model.predict(test_data)
for prediction, CHD in zip(predictions[:50], list(test_data)[0][1][:50]):
  print("Prediction of CGH: {:.2%}".format(prediction[0]) + " Actual Outcome: ",
        ("Postive" if bool(CHD) else "Negative"))

Prediction of CGH: 5.60% Actual Outcome:  Negative
Prediction of CGH: 6.63% Actual Outcome:  Postive
Prediction of CGH: 40.28% Actual Outcome:  Postive
Prediction of CGH: 29.44% Actual Outcome:  Negative
Prediction of CGH: 22.93% Actual Outcome:  Negative
Prediction of CGH: 70.20% Actual Outcome:  Negative
Prediction of CGH: 69.15% Actual Outcome:  Negative
Prediction of CGH: 28.10% Actual Outcome:  Negative
Prediction of CGH: 19.77% Actual Outcome:  Postive
Prediction of CGH: 69.15% Actual Outcome:  Negative
Prediction of CGH: 44.87% Actual Outcome:  Negative
Prediction of CGH: 0.19% Actual Outcome:  Negative
Prediction of CGH: 76.22% Actual Outcome:  Postive
Prediction of CGH: 35.09% Actual Outcome:  Postive
Prediction of CGH: 31.66% Actual Outcome:  Negative
Prediction of CGH: 41.19% Actual Outcome:  Negative
Prediction of CGH: 35.58% Actual Outcome:  Negative
Prediction of CGH: 84.82% Actual Outcome:  Postive
Prediction of CGH: 66.67% Actual Outcome:  Negative
Prediction of CGH: 71